#Implementing Decision Tress

In [2]:
import numpy as np
from collections import Counter

In [3]:
class Node:
  def __init__(self,feature = None,threshold=None ,left= None,right=None,*,value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right= right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

class Decision_Tree:
  def __init__(self, min_split_samples=2,max_depth=100,n_features=None):
    self.min_split_samples = min_split_samples
    self.max_depth = max_depth
    self.n_features= n_features
    self.root = None
    # we can also add n_features variable to the decision tree if we decide to use only
    # certain no of features and not all the features

  def get_leaf_value(self, y):
    labels=Counter(y)
    value = labels.most_common(1)[0][0]
    return value

  def get_splits(self,x,feature_id,threshold):
    lf_idx,rg_idx = [],[]
    for i in x[feature_id]:
      if i>threshold:
        lf_idx.append()



  def entropy(self, y):
    hist = np.bincount(y)
    p_class = hist/len(y)
    return -np.sum([p * np.log2(p) for p in p_class if p>0])

  def information_gain(self, x,y, threshold, feature_id):
    parent_ent = self.entropy(y)
    lf_idx = np.argwhere(x.iloc[:,feature_id] <= threshold).flatten()
    rg_idx = np.argwhere(x.iloc[:,feature_id] > threshold).flatten()
    if (len(lf_idx) == 0 or len(rg_idx) == 0):
            return 0
    lf_entropy = self.entropy(y.iloc[lf_idx])
    rg_entropy= self.entropy(y.iloc[rg_idx])
    gain = parent_ent - (len(lf_idx)*lf_entropy + len(rg_idx)* rg_entropy )/ len(y)
    return gain



  def best_split(self,x,y,feature_idx):

    # go through each features of the training samples and get the unique values of each feature and establish them as a threshold
    # and determine the Information gain and find the feature_id and split_threshold for which we get the maximum gain
    best_gain = -1
    best_feature_id, best_split_threshold = None, None

    for feature_id in feature_idx:

      x_column = x.iloc[:,feature_id]
      thresholds = np.unique(x_column)
      for threshold in thresholds:
        gain = self.information_gain(x,y,threshold,feature_id)

        if gain > best_gain:
          best_gain = gain
          best_split_threshold = threshold
          best_feature_id = feature_id
    return best_feature_id, best_split_threshold







  def build_tree(self,x,y,depth=0):
    n_samples, n_feats = x.shape
    n_labels = len(np.unique(y))


    # First I will determine the stopping criteria of the tree to determine not to grow the tree any further
    if ( depth>=self.max_depth or n_samples <= self.min_split_samples or n_labels == 1 ):
      value = self.get_leaf_value(y)
      return Node(value = value)

    feature_idx = np.random.choice(n_feats,self.n_features,replace = False)

    # find the best feature and split threshold

    best_feature_id, best_split_threshold = self.best_split(x,y, feature_idx)

    # print("best feartuer and threshold",best_feature_id, best_split_threshold)
    # create the left and right child nodes

    lf_idx = np.argwhere(x.iloc[:,best_feature_id] <= best_split_threshold).flatten()
    rg_idx = np.argwhere(x.iloc[:,best_feature_id] > best_split_threshold).flatten()

    left = self.build_tree(x.iloc[lf_idx,:],y.iloc[lf_idx],depth+1)
    right = self.build_tree(x.iloc[rg_idx,:],y.iloc[rg_idx],depth+1)

    return Node(best_feature_id,best_split_threshold, left,right)

    # repeat until the stopping criteria meets



  def fit(self,x,y):

    n_samples, n_feats = x.shape
    n_labels = len(np.unique(y))
    # also checking the count of the desired features is not more than the available features in the training data
    self.n_features = x.shape[1] if not self.n_features else min(self.n_features,x.shape[1])

    # I have to build a decision tree for my training
    self.root = self.build_tree(x,y)


  def traversal_tree(self,x,root):
    if (root.is_leaf_node()):
      return root.value

    if(x[root.feature] <= root.threshold):
      # tranverse the left node
      return self.traversal_tree(x,root.left)

    else:
        # traverse the right node
        return self.traversal_tree(x,root.right)





  def predict(self,x):

    # in here we have to tranverse through the already built decision tree from the training data

    #y_predict = np.array([self.traversal_tree(row,self.root) for i,row in x.iterrows()])
    y_predict = x.apply(lambda row: self.traversal_tree(row, self.root), axis=1)
    return y_predict





In [4]:

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np


# Load the breast cancer dataset
data = datasets.load_breast_cancer()

# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)

# Add the target variable to the DataFrame
df['target'] = data.target

# Split the data into X (features) and y (target)
X = df.drop(columns=['target'])
y = df['target']

# Split the data into training and testing sets (here we only focus on training set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Decision_Tree(max_depth=10,min_split_samples=2)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print("Accuracy of the data",acc)


Accuracy of the data 0.9473684210526315


<ipython-input-3-7c6f19dfcbdc>:124: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(x[root.feature] <= root.threshold):


In [11]:
from sklearn.tree import DecisionTreeClassifier
sk_model = DecisionTreeClassifier(max_depth=10,min_samples_split=2)
sk_model.fit(X_train,y_train)
sk_predictions = sk_model.predict(X_test)
sk_acc = accuracy(y_test, sk_predictions)
print("Accuracy of the data",sk_acc)

Accuracy of the data 0.9298245614035088


# Implementing Random Forest

It is the sequential implementation of N decision trees where each decision tree is trained on m samples of the dataset.
The final classification output of the model is majority of the class that is predicted from the N decision trees.

The hyper parameters of Random Forest on top of the decision tree are N : no of decision trees to be used and selecting m samples that are used to train the model.

In [7]:
from collections import Counter
class Random_Forest():
  def __init__(self,N=7,m=300):
    self.N = N
    self.m = m

  def fit(self,x,y):
    self.model = []
    for i in range(0,self.N):
      random_state = np.random.randint(0,1000)
      random_indices = x.sample(n=self.m, random_state=random_state).index
      x_train = x.loc[random_indices]
      y_train = y.loc[random_indices]
      self.model.append( Decision_Tree())
      self.model[i].fit(x_train,y_train)


  def predict(self,x):
    output = []
    y_predictions = []
    for i in range(0,self.N):
      y_predictions.append(self.model[i].predict(x))
    df = pd.DataFrame(y_predictions)
    output = df.apply(lambda col : col.value_counts().idxmax(),axis = 0)
    return output



In [8]:

Rd_fr = Random_Forest()
Rd_fr.fit(X_train, y_train)
predictions = Rd_fr.predict(X_test)


acc = accuracy(y_test, predictions)
print("Accuracy of the data",acc)


<ipython-input-3-7c6f19dfcbdc>:124: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(x[root.feature] <= root.threshold):


Accuracy of the data 0.956140350877193


In [10]:
from sklearn.ensemble import RandomForestClassifier

sk_rd_fr = RandomForestClassifier(n_estimators=7,max_samples=300)
sk_rd_fr.fit(X_train,y_train)
sk_predictions = sk_rd_fr.predict(X_test)
acc = accuracy(y_test, sk_predictions)
print("Accuracy of the data",acc)


Accuracy of the data 0.9649122807017544
